# Training Classification Models

In [1]:
!pip install torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0
!pip install pandas==1.3.5
!pip install Pillow==9.2.0
!pip install albumentations==1.3.0
!pip install opencv-python==4.7.0.72


In [2]:
!pip install scikit-learn segmentation-models-pytorch

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 114.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 78.1 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16425 sha256=b2a5165288aaa44f7daf058efbced20a3229021bfe9ff4a9caf41a8097106168
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=b0d78e7230d7c2613526429360dae768b77896317bc9dd89acff5db59db0cf68
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pyto

In [3]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

In [4]:
from google.colab import drive, files
drive.mount('/content/drive')

metadata_train = pd.read_csv("/content/drive/MyDrive/melanoma-segmentation-and-classification/data/metadata/ISIC-2017_Training_Data.csv")
metadata_validation = pd.read_csv("/content/drive/MyDrive/melanoma-segmentation-and-classification/data/metadata/ISIC-2017_Validation_Data.csv")
base_dir = "/content/drive/MyDrive/melanoma-segmentation-and-classification/data/"
image_folder_train= "ISIC-2017_Training_Data"
gt_folder_train="ISIC-2017_Training_Part1_GroundTruth"
image_folder_val= "ISIC-2017_Validation_Data"
gt_folder_val="ISIC-2017_Validation_Part1_GroundTruth"

img_pt_train = os.path.join(base_dir, image_folder_train)
gt_pt_train = os.path.join(base_dir, gt_folder_train)
img_pt_val = os.path.join(base_dir, image_folder_val)
gt_pt_val = os.path.join(base_dir, gt_folder_val)

metadata_train.head()

Mounted at /content/drive


,image_id,melanoma,age_approximate,sex
0,ISIC_0000000,0,0.6111,0
1,ISIC_0000001,0,0.3333,0
2,ISIC_0000002,1,0.6667,0
3,ISIC_0000003,0,0.3333,1
4,ISIC_0000004,1,0.8889,1


In [6]:


train_aug = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Transpose(p=0.5),
    A.Rotate(limit=30, p=0.5),
    A.RandomBrightnessContrast(p=0.75),
    A.OneOf([
            A.MotionBlur(blur_limit=5),
            A.MedianBlur(blur_limit=5),
        ], p=0.6),
    A.ColorJitter(brightness=0.2, contrast=0.2, p=0.5),
    A.CLAHE(clip_limit=4.0, p=0.7),
    A.HueSaturationValue(hue_shift_limit=15, sat_shift_limit=30, val_shift_limit=10, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=20, border_mode=0, p=0.6),
    A.Resize(224, 224),
    A.Normalize(),
    ToTensorV2()

], additional_targets={'mask': 'mask'})

val_aug = A.Compose([
    A.Resize(224, 224),
    A.Normalize(),
    ToTensorV2()
])
class MelanomaDataset(Dataset):
    def __init__(self, metadata, img_dir, mask_dir, augmentations=None, mode="image"):
        """
        Args:
            metadata: DataFrame containing image IDs, labels, and metadata.
            img_dir: Directory containing images.
            mask_dir: Directory containing segmentation masks.
            augmentations: Albumentations Compose object for data augmentation.
            mode: "image", "mask", or "feature" to control output for different models.
        """
        self.metadata = metadata
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.augmentations = augmentations
        self.mode = mode  # 'image', 'mask', or 'feature'

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        row = self.metadata.iloc[idx]
        image_id = row['image_id']
        label = row['melanoma']

        # Load the image
        img_path = os.path.join(self.img_dir, f"{image_id}.jpg")
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

        # Load the mask
        mask_path = os.path.join(self.mask_dir, f"{image_id}_segmentation.png")
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Load mask as grayscale

        # Apply augmentations
        if self.augmentations:
            augmented = self.augmentations(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        # Metadata processing
        metadata_t = row.drop(['image_id', 'melanoma']).values.astype(float)
        metadata = torch.tensor(metadata_t, dtype=torch.float32)

        if self.mode == "image":
            return image.float(), label
        elif self.mode == "mask":
            return mask, label
        elif self.mode == "feature":
            return image.float(), mask, metadata, torch.tensor(label, dtype=torch.long)
        else:
            raise ValueError("Invalid mode. Use 'image', 'mask', or 'feature'.")
FutureWarning

FutureWarning

In [ ]:
feature_train_dataset = MelanomaDataset(
    metadata=metadata_train,
    img_dir=img_pt_train,
    mask_dir=gt_pt_train,
    augmentations=val_aug,
    mode="feature"
)

feature_val_dataset = MelanomaDataset(
    metadata=metadata_validation,
    img_dir=img_pt_val,
    mask_dir=gt_pt_val,
    augmentations=val_aug,
    mode="feature"
)

# Data loader for feature extraction
feature_train_loader = DataLoader(feature_train_dataset, batch_size=16, shuffle=False)
feature_val_loader = DataLoader(feature_val_dataset, batch_size=16, shuffle=False)

In [11]:
#CUDA enable if not cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
def pad_metadata(metadata_list, batch_size):
    padded_metadata = []
    for metadata in metadata_list:
        if metadata.shape[0] < batch_size:
            # Pad the metadata to match the batch size
            padding = np.zeros((batch_size - metadata.shape[0], metadata.shape[1]))
            metadata = np.vstack((metadata, padding))
        padded_metadata.append(metadata)
    return np.array(padded_metadata)

In [ ]:
from torch import nn
from torchvision import models
import torch
from segmentation_models_pytorch import Unet  # Assuming you use smp for U-Net

# Define two U-Nets: one for image, one for mask
class UNetForImage(nn.Module):
    def __init__(self, in_channels=3, num_classes=1):
        super(UNetForImage, self).__init__()
        self.unet = Unet(encoder_name="resnet50", in_channels=in_channels, classes=num_classes)

    def forward(self, x):
        return self.unet(x)

class UNetForMask(nn.Module):
    def __init__(self, in_channels=1, num_classes=1):
        super(UNetForMask, self).__init__()
        self.unet = Unet(encoder_name="resnet50", in_channels=in_channels, classes=num_classes)

    def forward(self, x):
        return self.unet(x)

# Ensemble model combining both U-Nets
class EnsembleModel(nn.Module):
    def __init__(self, unet_image, unet_mask):
        super(EnsembleModel, self).__init__()
        self.unet_image = unet_image
        self.unet_mask = unet_mask
        self.global_pool = nn.AdaptiveAvgPool2d(1)  # Pool to a single value per channel
        self.fc = nn.Linear(2, 1)  # Combine two global predictions into one score

    def forward(self, image, mask):
        # Forward pass through U-Nets
        img_pred = self.unet_image(image)  # Output: [B, 1, H, W]
        mask_pred = self.unet_mask(mask)  # Output: [B, 1, H, W]

        # Global pooling to reduce to [B, 1, 1, 1]
        img_global = self.global_pool(img_pred).view(img_pred.size(0), -1)  # Output: [B, 1]
        mask_global = self.global_pool(mask_pred).view(mask_pred.size(0), -1)  # Output: [B, 1]

        # Combine predictions
        combined = torch.cat([img_global, mask_global], dim=1)  # Output: [B, 2]
        return self.fc(combined).squeeze(1)

# Initialize models
unet_image = UNetForImage()
unet_mask = UNetForMask()
ensemble_model = EnsembleModel(unet_image, unet_mask)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 288MB/s]


In [ ]:
# Loss function for binary classification/segmentation
criterion = nn.BCEWithLogitsLoss()

# Optimizer
optimizer = torch.optim.Adam(ensemble_model.parameters(), lr=1e-4)

# Device setup
device = "cuda" if torch.cuda.is_available() else "cpu"
ensemble_model.to(device)

num_epochs = 10

for epoch in range(num_epochs):
    ensemble_model.train()
    epoch_loss = 0.0

    for images, masks, metadata, targets in feature_train_loader:
        images = images.to(device).float()  # Image input: [B, 3, H, W]
        masks = masks.to(device).unsqueeze(1).float()    # Mask input: [B, 1, H, W]
        targets = targets.to(device).float()  # Targets: Scalar or [B]
        # Forward pass through the ensemble model
        outputs = ensemble_model(images, masks)  # Expected shape: [B, 1, H, W]

        # Compute loss
        loss = criterion(outputs, targets)
        epoch_loss += loss.item()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")



Epoch [1/10], Loss: 49.1928
Epoch [2/10], Loss: 45.6882
Epoch [3/10], Loss: 41.8022
Epoch [4/10], Loss: 38.9118
Epoch [5/10], Loss: 35.5439
Epoch [6/10], Loss: 33.4095
Epoch [7/10], Loss: 29.6392
Epoch [8/10], Loss: 27.5142
Epoch [9/10], Loss: 23.4562
Epoch [10/10], Loss: 20.7486


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Track the best validation loss
best_val_loss = float('inf')
best_model_path = "best_model.pth"

# Validation Loop
for epoch in range(num_epochs):
    ensemble_model.train()
    epoch_loss = 0.0

    for images, masks, metadata, targets in feature_train_loader:
        images = images.to(device).float()  # Image input: [B, 3, H, W]
        masks = masks.to(device).unsqueeze(1).float()  # Mask input: [B, 1, H, W]
        targets = targets.to(device).float()  # Targets: [B]

        # Forward pass through the ensemble model
        outputs = ensemble_model(images, masks)  # Expected shape: [B]
        loss = criterion(outputs, targets)
        epoch_loss += loss.item()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

    # Validation phase
    ensemble_model.eval()
    val_loss = 0.0
    all_targets = []
    all_predictions = []

    with torch.no_grad():
        for images, masks, metadata, targets in feature_val_loader:
            images = images.to(device).float()
            masks = masks.to(device).unsqueeze(1).float()
            targets = targets.to(device).float()

            outputs = ensemble_model(images, masks)  # Output: [B]
            predictions = (torch.sigmoid(outputs) > 0.5).float()  # Convert logits to binary predictions

            # Collect results for metrics
            all_targets.extend(targets.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())

            # Compute validation loss
            loss = criterion(outputs, targets)
            val_loss += loss.item()

    val_loss /= len(feature_val_loader)

    # Compute and print classification metrics
    print(f"Validation Loss: {val_loss:.4f}")
    print("Classification Report:")
    print(classification_report(all_targets, all_predictions))

    # Save the best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(ensemble_model.state_dict(), best_model_path)
        print(f"Best model saved with validation loss: {best_val_loss:.4f}")


Epoch [1/10], Loss: 19.0546
Validation Loss: 0.5104
Classification Report:
              precision    recall  f1-score   support

         0.0       0.81      0.90      0.85       119
         1.0       0.29      0.17      0.21        30

    accuracy                           0.75       149
   macro avg       0.55      0.53      0.53       149
weighted avg       0.71      0.75      0.72       149

Best model saved with validation loss: 0.5104
Epoch [2/10], Loss: 17.1035
Validation Loss: 0.5243
Classification Report:
              precision    recall  f1-score   support

         0.0       0.80      1.00      0.89       119
         1.0       0.00      0.00      0.00        30

    accuracy                           0.80       149
   macro avg       0.40      0.50      0.44       149
weighted avg       0.64      0.80      0.71       149



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [3/10], Loss: 16.8074
Validation Loss: 0.6029
Classification Report:
              precision    recall  f1-score   support

         0.0       0.80      0.99      0.89       119
         1.0       0.50      0.03      0.06        30

    accuracy                           0.80       149
   macro avg       0.65      0.51      0.47       149
weighted avg       0.74      0.80      0.72       149

Epoch [4/10], Loss: 15.0590
Validation Loss: 0.5902
Classification Report:
              precision    recall  f1-score   support

         0.0       0.80      1.00      0.89       119
         1.0       1.00      0.03      0.06        30

    accuracy                           0.81       149
   macro avg       0.90      0.52      0.48       149
weighted avg       0.84      0.81      0.72       149

Epoch [5/10], Loss: 13.9452
Validation Loss: 0.6202
Classification Report:
              precision    recall  f1-score   support

         0.0       0.80      0.99      0.89       119
         1.0

In [ ]:
files.download(best_model_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import torch
import torch.nn as nn
from torchvision import models


# Collect metadata as a list
train_metadata_list = []
val_metadata_list = []

for _, _, metadata, _ in feature_train_loader:
    train_metadata_list.append(metadata.numpy())

for _, _, metadata, _ in feature_val_loader:
    val_metadata_list.append(metadata.numpy())


for idx, metadata in enumerate(val_metadata_list):
    print(f"Metadata {idx} Shape: {metadata.shape}")

batch_size = 16
train_metadata_padded = pad_metadata(train_metadata_list, batch_size)
val_metadata_padded = pad_metadata(val_metadata_list, batch_size)



# Check for uniformity in metadata shapes
train_metadata = np.concatenate([m.flatten() for m in train_metadata_padded], axis=0)
val_metadata = np.concatenate([m.flatten() for m in val_metadata_padded], axis=0)

print(f"Train Metadata Shape: {train_metadata.shape}")
print(f"Validation Metadata Shape: {val_metadata.shape}")




# Feature extraction class using ResNet
class FeatureExtractor(nn.Module):
    def __init__(self, backbone):
        super(FeatureExtractor, self).__init__()
        self.backbone = nn.Sequential(*list(backbone.children())[:-1])  # Remove the final FC layer

    def forward(self, x):
        features = self.backbone(x)
        return features.view(features.size(0), -1)  # Flatten features

# Initialize feature extractors
resnet_image = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
resnet_mask = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
image_feature_extractor = FeatureExtractor(resnet_image)
mask_feature_extractor = FeatureExtractor(resnet_mask)

train_metadata = train_metadata.reshape(-1, 3)  # Reshape to 2D (n_samples, 1)
val_metadata = val_metadata.reshape(-1, 3)

valid_indices_val = ~np.all(val_metadata == 0, axis=1)

val_metadata = val_metadata[valid_indices_val]

# Function to extract features
def extract_features(loader, model, device, is_mask=False):
    model.to(device)
    model.eval()
    features_list = []
    labels_list = []
    with torch.no_grad():
        for image, mask, metadata, label in loader:
            inputs = mask if is_mask else image  # Choose input based on type
            if is_mask:
                # Convert single-channel masks to 3-channel by repeating along the channel dimension
                inputs = inputs.unsqueeze(1) if len(inputs.shape) == 3 else inputs  # Ensure [B, C, H, W]
                inputs = inputs.repeat(1, 3, 1, 1)  # Repeat along channel dimension to create 3 channels
            inputs = inputs.to(device).float() / 255.0  # Convert to float32 and normalize to [0, 1]
            features = model(inputs)
            features_list.append(features.cpu().numpy())
            labels_list.append(label.numpy())
    return np.concatenate(features_list, axis=0), np.concatenate(labels_list, axis=0)



device = "cuda" if torch.cuda.is_available() else "cpu"

# Extract features from images
image_features_train, y_train = extract_features(feature_train_loader, image_feature_extractor, device, is_mask=False)
image_features_val, y_val = extract_features(feature_val_loader, image_feature_extractor, device, is_mask=False)

# Extract features from masks
mask_features_train, _ = extract_features(feature_train_loader, mask_feature_extractor, device, is_mask=True)
mask_features_val, _ = extract_features(feature_val_loader, mask_feature_extractor, device, is_mask=True)

# Combine features with metadata


combined_features_train = np.concatenate([image_features_train, mask_features_train, train_metadata], axis=1)
combined_features_val = np.concatenate([image_features_val, mask_features_val, val_metadata], axis=1)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(combined_features_train)
X_val_scaled = scaler.transform(combined_features_val)

# Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Evaluate the mode
y_pred = rf_model.predict(X_val_scaled)
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy:.4f}")


Metadata 0 Shape: (16, 3)
Metadata 1 Shape: (16, 3)
Metadata 2 Shape: (16, 3)
Metadata 3 Shape: (16, 3)
Metadata 4 Shape: (16, 3)
Metadata 5 Shape: (16, 3)
Metadata 6 Shape: (16, 3)
Metadata 7 Shape: (16, 3)
Metadata 8 Shape: (16, 3)
Metadata 9 Shape: (5, 3)
Train Metadata Shape: (5232,)
Validation Metadata Shape: (480,)
Validation Accuracy: 0.8121


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Generate predictions
y_pred = rf_model.predict(X_val_scaled)

# Classification report
print("Classification Report:")
print(classification_report(y_val, y_pred))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.81      1.00      0.89       119
           1       1.00      0.07      0.12        30

    accuracy                           0.81       149
   macro avg       0.90      0.53      0.51       149
weighted avg       0.85      0.81      0.74       149

Confusion Matrix:
[[119   0]
 [ 28   2]]


In [ ]:
import numpy as np
from sklearn.model_selection import GridSearchCV

# Dynamically generate parameter ranges
param_grid = {
    'n_estimators': list(range(50, 201, 50)),  # Custom step
    'max_depth': list(range(10, 31, 10)),      # Custom step
    'min_samples_split': list(range(2, 11, 3)),  # Custom step
    'min_samples_leaf': list(range(1, 5, 1))  # Custom step
}

# Perform GridSearchCV
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=3,                     # 3-fold cross-validation
    scoring='accuracy',
    verbose=2,
    n_jobs=-1                 # Use all processors
)

grid_search.fit(X_train_scaled, y_train)

# Best hyperparameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)


Fitting 3 folds for each of 144 candidates, totalling 432 fits
Best Parameters: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best Cross-Validation Score: 0.8279854814446791


In [ ]:

#Save rf_model
import pickle
with open('rf_model.pkl', 'wb') as file:
    pickle.dump(rf_model, file)

#Download rf_model.pkl
files.download('rf_model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
best_params = grid_search.best_params_

# Train a new Random Forest with the best parameters
best_rf_model = RandomForestClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=42
)
best_rf_model.fit(X_train_scaled, y_train)

# Predict on the validation set
y_pred_best = best_rf_model.predict(X_val_scaled)

# Evaluate the model
print("Validation Accuracy:", accuracy_score(y_val, y_pred_best))

print("\nClassification Report:")
print(classification_report(y_val, y_pred_best))

print("\nConfusion Matrix:")
print(confusion_matrix(y_val, y_pred_best))

Validation Accuracy: 0.8187919463087249

Classification Report:
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       119
           1       1.00      0.10      0.18        30

    accuracy                           0.82       149
   macro avg       0.91      0.55      0.54       149
weighted avg       0.85      0.82      0.75       149


Confusion Matrix:
[[119   0]
 [ 27   3]]


# Just the image

In [5]:
import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18
from torch import nn, optim
from sklearn.metrics import classification_report
import albumentations as A
from albumentations.pytorch import ToTensorV2

metadata_train = pd.read_csv("/content/drive/MyDrive/melanoma-segmentation-and-classification/data/metadata/ISIC-2017_Training_Data.csv")
metadata_validation = pd.read_csv("/content/drive/MyDrive/melanoma-segmentation-and-classification/data/metadata/ISIC-2017_Validation_Data.csv")

# Dataset Class
class MelanomaDataset2(Dataset):
    def __init__(self, metadata, img_dir, augmentations=None):
        """
        Args:
            metadata: DataFrame containing image IDs and labels.
            img_dir: Directory containing images.
            augmentations: Albumentations Compose object for data augmentation.
        """
        self.metadata = metadata
        self.img_dir = img_dir
        self.augmentations = augmentations

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        row = self.metadata.iloc[idx]
        image_id = row['image_id']
        label = row['melanoma']

        # Load the image
        img_path = os.path.join(self.img_dir, f"{image_id}.jpg")
        image = cv2.imread(img_path)
        if image is None:
            raise FileNotFoundError(f"Image file not found: {img_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

        # Apply augmentations
        if self.augmentations:
            augmented = self.augmentations(image=image)
            image = augmented['image']

        # Convert image to tensor


        return image.float(), torch.tensor(label, dtype=torch.float32)


# Augmentations
train_aug = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Transpose(p=0.5),
    A.Rotate(limit=30, p=0.5),
    A.RandomBrightnessContrast(p=0.75),
    A.OneOf([
            A.MotionBlur(blur_limit=5),
            A.MedianBlur(blur_limit=5),
        ], p=0.6),
    A.ColorJitter(brightness=0.2, contrast=0.2, p=0.5),
    A.CLAHE(clip_limit=4.0, p=0.7),
    A.HueSaturationValue(hue_shift_limit=15, sat_shift_limit=30, val_shift_limit=10, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=20, border_mode=0, p=0.6),
    A.Resize(224, 224),
    A.Normalize(),
    ToTensorV2()

], additional_targets={'mask': 'mask'})

val_aug = A.Compose([
    A.Resize(224, 224),
    A.Normalize(),
    ToTensorV2()
])

# Dataset and DataLoader
train_dataset_2 = MelanomaDataset2(metadata=metadata_train, img_dir=img_pt_train, augmentations=train_aug)
val_dataset_2 = MelanomaDataset2(metadata=metadata_validation, img_dir=img_pt_val, augmentations=val_aug)

train_loader_2 = DataLoader(train_dataset_2, batch_size=16, shuffle=True)
val_loader_2 = DataLoader(val_dataset_2, batch_size=16, shuffle=False)

In [21]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device="cuda"):
    model.to(device)

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print("-" * 30)

        # Training phase
        model.train()
        train_loss = 0.0
        correct_train = 0
        total_train = 0

        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device).unsqueeze(1)  # Ensure [B, 1]

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            preds = (outputs > 0.5).float()
            correct_train += (preds == labels).sum().item()
            total_train += labels.size(0)

        train_loss /= total_train
        train_accuracy = correct_train / total_train

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct_val = 0
        total_val = 0
        all_targets = []
        all_predictions = []

        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device).float().unsqueeze(1)

                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * images.size(0)
                preds = (outputs > 0.5).float()
                correct_val += (preds == labels).sum().item()
                total_val += labels.size(0)

                # Store all predictions and targets for classification report
                all_targets.extend(labels.cpu().numpy())
                all_predictions.extend(preds.cpu().numpy())

        val_loss /= total_val
        val_accuracy = correct_val / total_val

        print(all_targets)
        print(all_predictions)

        # Generate classification report
        report = classification_report(all_targets, all_predictions, target_names=["Class 0", "Class 1"], digits=4)

        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
        print("Validation Classification Report:")
        print(report)
        print()

    return model




## ResNet18



In [22]:
from torchvision.models import resnet18
from torch import optim

# Load pretrained Resnet-18
resnet = resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)
# Modify the classifier
num_features = resnet.fc.in_features
resnet.fc = nn.Sequential(
    nn.Linear(num_features, 1),
    nn.Sigmoid()
)

# Loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.001)

# Train Resnet-18
trained_resnet18 = train_model(resnet, train_loader_2, val_loader_2, criterion, optimizer, num_epochs=10, device=device)
torch.save(trained_resnet18.state_dict(), "resnet18_melanoma.pth")
files.download("resnet18_melanoma.pth")


Epoch 1/10
------------------------------
[array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=f

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=f

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=f

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## EfficientNet-B0

In [23]:
from torchvision.models import efficientnet_b0

# Load pretrained EfficientNet-B0
effnet = efficientnet_b0(weights=torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1)
# Modify the classifier
num_features = effnet.classifier[1].in_features
effnet.classifier = nn.Sequential(
    nn.Linear(num_features, 1),
    nn.Sigmoid()
)

# Loss function and optimizer
optimizer = optim.Adam(effnet.parameters(), lr=0.001)

# Train EfficientNet-B0
trained_effnet = train_model(effnet, train_loader_2, val_loader_2, criterion, optimizer, num_epochs=10, device=device)
torch.save(trained_effnet.state_dict(), "effnet_melanoma.pth")
files.download("effnet_melanoma.pth")


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 177MB/s]


Epoch 1/15
------------------------------
[array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=f

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=f

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=f

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## MobileNetV2

In [24]:
from torchvision.models import mobilenet_v2

# Load pretrained MobileNetV2
mobilenet = mobilenet_v2(weights=torchvision.models.MobileNet_V2_Weights.IMAGENET1K_V1)

# Modify the classifier
num_features = mobilenet.last_channel
mobilenet.classifier = nn.Sequential(
    nn.Linear(num_features, 1),
    nn.Sigmoid()
)

# Loss function and optimizer
optimizer = optim.Adam(mobilenet.parameters(), lr=0.001)

# Train MobileNetV2
trained_mobilenet = train_model(mobilenet, train_loader_2, val_loader_2, criterion, optimizer, num_epochs=10, device=device)


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 206MB/s]


Epoch 1/15
------------------------------
[array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=f

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=f

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([1.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=f

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
torch.save(trained_resnet18.state_dict(), "resnet18_melanoma.pth")
files.download("resnet18_melanoma.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
torch.save(mobilenet.state_dict(), "mobilenet_melanoma.pth")
files.download("mobilenet_melanoma.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Imágenes aplanadas

In [6]:
import numpy as np

class MelanomaDatasetFlattened(Dataset):
    def __init__(self, metadata, img_dir, augmentations=None):
        self.metadata = metadata
        self.img_dir = img_dir
        self.augmentations = augmentations

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        row = self.metadata.iloc[idx]
        image_id = row['image_id']
        label = row['melanoma']

        # Load the image
        img_path = os.path.join(self.img_dir, f"{image_id}.jpg")
        image = cv2.imread(img_path)
        if image is None:
            raise FileNotFoundError(f"Image file not found: {img_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

        # Apply augmentations
        if self.augmentations:
            augmented = self.augmentations(image=image)
            image = augmented['image']

        # Flatten the image
        image = image.reshape(-1)  # Flatten to 1D

        return image, label

# Load data
train_dataset_flat = MelanomaDatasetFlattened(metadata=metadata_train, img_dir=img_pt_train, augmentations=val_aug)
val_dataset_flat = MelanomaDatasetFlattened(metadata=metadata_validation, img_dir=img_pt_val, augmentations=val_aug)

# Collect all samples into arrays for SVM
def collect_dataset(dataset):
    images, labels = [], []
    for img, label in dataset:
        images.append(img)
        labels.append(label)
    return np.array(images), np.array(labels)

train_images, train_labels = collect_dataset(train_dataset_flat)
val_images, val_labels = collect_dataset(val_dataset_flat)


In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_images = scaler.fit_transform(train_images)
val_images = scaler.transform(val_images)

In [8]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Train the SVM
svm = SVC(kernel="linear", C=1, random_state=42)
svm.fit(train_images, train_labels)

# Predict on validation set
val_predictions = svm.predict(val_images)

# Classification report
report = classification_report(val_labels, val_predictions, target_names=["Non-Melanoma", "Melanoma"])
print("Validation Classification Report:")
print(report)


Validation Classification Report:
              precision    recall  f1-score   support

Non-Melanoma       0.83      0.87      0.85       120
    Melanoma       0.36      0.30      0.33        30

    accuracy                           0.75       150
   macro avg       0.60      0.58      0.59       150
weighted avg       0.74      0.75      0.74       150



In [9]:
import joblib

# Save the trained model
joblib.dump(svm, "svc_flattened.pkl")
files.download("svc_flattened.pkl")
print("Model saved as 'svc_flattened.pkl'")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Model saved as 'svc_flattened.pkl'


In [18]:
from xgboost import XGBClassifier

# Train XGBoost
xgb = XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)
xgb.fit(train_images, train_labels)

# Predict and Evaluate
val_predictions = xgb.predict(val_images)
print(classification_report(val_labels, val_predictions, target_names=["Non-Melanoma", "Melanoma"]))


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [07:45:36] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

Non-Melanoma       0.81      1.00      0.90       120
    Melanoma       1.00      0.07      0.12        30

    accuracy                           0.81       150
   macro avg       0.91      0.53      0.51       150
weighted avg       0.85      0.81      0.74       150



In [19]:
from sklearn.ensemble import RandomForestClassifier

# Train Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(train_images, train_labels)

# Predict and Evaluate
val_predictions = rf.predict(val_images)
print(classification_report(val_labels, val_predictions, target_names=["Non-Melanoma", "Melanoma"]))


              precision    recall  f1-score   support

Non-Melanoma       0.80      0.96      0.87       120
    Melanoma       0.29      0.07      0.11        30

    accuracy                           0.78       150
   macro avg       0.54      0.51      0.49       150
weighted avg       0.70      0.78      0.72       150



In [14]:
from torchvision.models import efficientnet_b0
# Training Loop Function
import time
from sklearn.metrics import classification_report, roc_auc_score
from torch.cuda.amp import GradScaler, autocast
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm


# Load pretrained EfficientNet-B0
effnet = efficientnet_b0(weights=torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1)

class CustomEfficientNet(nn.Module):
    def __init__(self, out_dim, pretrained=True):
        super(CustomEfficientNet, self).__init__()
        self.backbone = efficientnet_b0(
            weights=torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1 if pretrained else None
        )
        in_features = self.backbone.classifier[1].in_features

        # Custom classifier head with Dropouts
        self.backbone.classifier = nn.Identity()  # Remove the default classifier
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(in_features, out_dim)

    def forward(self, x):
        x = self.backbone(x)
        x = self.dropout(x)
        x = self.fc(x)
        return x


# Define Model
model = CustomEfficientNet(out_dim=1)  # Single output for binary classification

# Loss Function and Optimizer
criterion = nn.BCEWithLogitsLoss()  # Better for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)


def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device="cuda"):
    model.to(device)

    # Gradient scaler for mixed precision
    scaler = GradScaler()

    # Scheduler (cosine annealing warm restarts)
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=1e-6)

    best_val_auc = 0.0
    best_model_wts = None

    for epoch in range(1, num_epochs + 1):
        print(f"Epoch {epoch}/{num_epochs}")
        print("-" * 30)

        # Training phase
        model.train()
        train_loss = 0.0
        correct_train = 0
        total_train = 0
        all_train_targets = []
        all_train_predictions = []

        for images, labels in tqdm(train_loader, desc="Training"):
            images = images.to(device)
            labels = labels.to(device).unsqueeze(1)  # [B, 1]

            optimizer.zero_grad()

            with autocast():  # Mixed precision training
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item() * images.size(0)
            preds = (outputs > 0.5).float()
            correct_train += (preds == labels).sum().item()
            total_train += labels.size(0)
            all_train_targets.extend(labels.cpu().numpy())
            all_train_predictions.extend(preds.cpu().numpy())

        train_loss /= total_train
        train_accuracy = correct_train / total_train

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct_val = 0
        total_val = 0
        all_val_targets = []
        all_val_predictions = []

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc="Validation"):
                images = images.to(device)
                labels = labels.to(device).unsqueeze(1)  # [B, 1]

                with autocast():
                    outputs = model(images)
                    loss = criterion(outputs, labels)

                val_loss += loss.item() * images.size(0)
                preds = (outputs > 0.5).float()
                correct_val += (preds == labels).sum().item()
                total_val += labels.size(0)
                all_val_targets.extend(labels.cpu().numpy())
                all_val_predictions.extend(preds.cpu().numpy())

        val_loss /= total_val
        val_accuracy = correct_val / total_val

        # Calculate AUC for validation
        val_auc = roc_auc_score(all_val_targets, all_val_predictions)

        # Print metrics
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val AUC: {val_auc:.4f}")

        # Classification report
        report = classification_report(all_val_targets, all_val_predictions, target_names=["Class 0", "Class 1"], digits=4)
        print("Validation Classification Report:")
        print(report)

        # Scheduler step
        scheduler.step()

        # Save the best model
        if val_auc > best_val_auc:
            best_val_auc = val_auc
            best_model_wts = model.state_dict()
            print("Best model updated.")

    # Load best model weights
    if best_model_wts is not None:
        model.load_state_dict(best_model_wts)
    return model


In [17]:

# Dataset and DataLoader
train_dataset_2 = MelanomaDataset2(metadata=metadata_train, img_dir=img_pt_train, augmentations=train_aug)
val_dataset_2 = MelanomaDataset2(metadata=metadata_validation, img_dir=img_pt_val, augmentations=val_aug)

train_loader_2 = DataLoader(train_dataset_2, batch_size=16, shuffle=True)
val_loader_2 = DataLoader(val_dataset_2, batch_size=16, shuffle=False)

# Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trained_model = train_model(model, train_loader_2, val_loader_2, criterion, optimizer, num_epochs=10, device=device)

# Save Model
torch.save(trained_model.state_dict(), "effnet_melanoma.pth")
files.download("effnet_melanoma.pth")

Epoch 1/10
------------------------------


Validation: 100%|██████████| 10/10 [02:56<00:00, 17.69s/it]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

Train Loss: 0.4657, Train Accuracy: 0.8140
Val Loss: 0.5606, Val Accuracy: 0.8000, Val AUC: 0.5000
Validation Classification Report:
              precision    recall  f1-score   support

     Class 0     0.8000    1.0000    0.8889       120
     Class 1     0.0000    0.0000    0.0000        30

    accuracy                         0.8000       150
   macro avg     0.4000    0.5000    0.4444       150
weighted avg     0.6400    0.8000    0.7111       150

Best model updated.
Epoch 2/10
------------------------------


Validation: 100%|██████████| 10/10 [00:48<00:00,  4.83s/it]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

Train Loss: 0.4594, Train Accuracy: 0.8140
Val Loss: 0.4895, Val Accuracy: 0.8000, Val AUC: 0.5000
Validation Classification Report:
              precision    recall  f1-score   support

     Class 0     0.8000    1.0000    0.8889       120
     Class 1     0.0000    0.0000    0.0000        30

    accuracy                         0.8000       150
   macro avg     0.4000    0.5000    0.4444       150
weighted avg     0.6400    0.8000    0.7111       150

Epoch 3/10
------------------------------


...

Train Loss: 0.3515, Train Accuracy: 0.8415
Val Loss: 0.3773, Val Accuracy: 0.8200, Val AUC: 0.5625
Validation Classification Report:
              precision    recall  f1-score   support

     Class 0     0.8207    0.9917    0.8981       120
     Class 1     0.8000    0.1333    0.2286        30

    accuracy                         0.8200       150
   macro avg     0.8103    0.5625    0.5633       150
weighted avg     0.8166    0.8200    0.7642       150

Epoch 10/10
------------------------------


Validation: 100%|██████████| 10/10 [00:48<00:00,  4.85s/it]


Train Loss: 0.3426, Train Accuracy: 0.8490
Val Loss: 0.3694, Val Accuracy: 0.8200, Val AUC: 0.5750
Validation Classification Report:
              precision    recall  f1-score   support

     Class 0     0.8252    0.9833    0.8973       120
     Class 1     0.7143    0.1667    0.2703        30

    accuracy                         0.8200       150
   macro avg     0.7697    0.5750    0.5838       150
weighted avg     0.8030    0.8200    0.7719       150

Best model updated.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Feature extractor + SVC or Random Forest



In [10]:
import torch
import torch.nn as nn
from torchvision import models
from torchvision.models import ResNet50_Weights, VGG16_Weights

# Load ResNet50
resnet50 = models.resnet50(weights=ResNet50_Weights.DEFAULT)
resnet50 = nn.Sequential(*list(resnet50.children())[:-1])  # Remove the fc layer (classifier)
resnet50.eval()

# Load VGG16
vgg16 = models.vgg16(weights=VGG16_Weights.DEFAULT)
vgg16 = nn.Sequential(*list(vgg16.features))  # Use only the feature extractor
vgg16.eval()


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 218MB/s]
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 238MB/s]


Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [11]:
from torch.utils.data import DataLoader
from torchvision.transforms import Normalize, ToTensor, Resize, Compose
# Define preprocessing transformations
preprocess = A.Compose([
    A.Resize(224, 224),  # Resize to 224x224
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet normalization
    ToTensorV2()  # Convert to PyTorch tensors
])

# Modify the dataset class to use PyTorch transforms
class MelanomaDataset3(Dataset):
    def __init__(self, metadata, img_dir, transform=None):
        self.metadata = metadata
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        row = self.metadata.iloc[idx]
        image_id = row['image_id']

        # Load the image
        img_path = os.path.join(self.img_dir, f"{image_id}.jpg")
        image = cv2.imread(img_path)
        if image is None:
            raise FileNotFoundError(f"Image file not found: {img_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Apply transformations
        if self.transform:
            transformed = self.transform(image=image)
            image = transformed['image']

        return image.float()

# Load datasets
train_dataset = MelanomaDataset3(metadata_train, img_pt_train, transform=preprocess)
val_dataset = MelanomaDataset3(metadata_validation, img_pt_val, transform=preprocess)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [12]:
from tqdm import tqdm
import numpy as np
import torch

# Function to extract features with a progress bar
def extract_features(loader, model, device="cpu"):
    model.to(device)  # Ensure the model is on the correct device
    features_list = []

    # Add tqdm progress bar to the DataLoader loop
    with torch.no_grad():
        for images in tqdm(loader, desc="Extracting Features"):
            images = images.to(device)  # Move input images to the same device as the model
            features = model(images)  # Forward pass through the model
            features = features.view(features.size(0), -1)  # Flatten the features
            features_list.append(features.cpu().numpy())  # Move features to CPU and convert to NumPy

    return np.vstack(features_list)  # Combine all features into a single array

# Extract features for training and validation sets
device = "cuda" if torch.cuda.is_available() else "cpu"

# ResNet50 features
print("Extracting features from ResNet50...")
train_features_resnet = extract_features(train_loader, resnet50, device)
val_features_resnet = extract_features(val_loader, resnet50, device)

# VGG16 features
print("Extracting features from VGG16...")

train_features_vgg = extract_features(train_loader, vgg16, device)
val_features_vgg = extract_features(val_loader, vgg16, device)


Extracting features from ResNet50...


Extracting Features: 100%|██████████| 5/5 [00:32<00:00,  6.54s/it]


Extracting features from VGG16...


Extracting Features: 100%|██████████| 5/5 [00:33<00:00,  6.68s/it]


In [52]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# SVC with ResNet50 features
svc_resnet = SVC(kernel="linear", random_state=42)
svc_resnet.fit(train_features_resnet, metadata_train['melanoma'])
val_predictions_svc_resnet = svc_resnet.predict(val_features_resnet)
print("SVC (ResNet50) Classification Report:")
print(classification_report(metadata_validation['melanoma'], val_predictions_svc_resnet))

# SVC with VGG16 features
svc_vgg = SVC(kernel="linear", random_state=42)
svc_vgg.fit(train_features_vgg, metadata_train['melanoma'])
val_predictions_svc_vgg = svc_vgg.predict(val_features_vgg)
print("SVC (VGG16) Classification Report:")
print(classification_report(metadata_validation['melanoma'], val_predictions_svc_vgg))

# Random Forest with ResNet50 features
rf_resnet = RandomForestClassifier(n_estimators=100, random_state=42)
rf_resnet.fit(train_features_resnet, metadata_train['melanoma'])
val_predictions_rf_resnet = rf_resnet.predict(val_features_resnet)
print("Random Forest (ResNet50) Classification Report:")
print(classification_report(metadata_validation['melanoma'], val_predictions_rf_resnet))

# Random Forest with VGG16 features
rf_vgg = RandomForestClassifier(n_estimators=100, random_state=42)
rf_vgg.fit(train_features_vgg, metadata_train['melanoma'])
val_predictions_rf_vgg = rf_vgg.predict(val_features_vgg)
print("Random Forest (VGG16) Classification Report:")
print(classification_report(metadata_validation['melanoma'], val_predictions_rf_vgg))


SVC (ResNet50) Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.82      0.81       120
           1       0.24      0.23      0.24        30

    accuracy                           0.70       150
   macro avg       0.53      0.53      0.53       150
weighted avg       0.70      0.70      0.70       150

SVC (VGG16) Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.90      0.86       120
           1       0.37      0.23      0.29        30

    accuracy                           0.77       150
   macro avg       0.60      0.57      0.57       150
weighted avg       0.73      0.77      0.75       150

Random Forest (ResNet50) Classification Report:
              precision    recall  f1-score   support

           0       0.80      1.00      0.89       120
           1       0.00      0.00      0.00        30

    accuracy                           0.80       150
   macr

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Random Forest (VGG16) Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.98      0.88       120
           1       0.33      0.03      0.06        30

    accuracy                           0.79       150
   macro avg       0.57      0.51      0.47       150
weighted avg       0.71      0.79      0.72       150



In [13]:
!pip install timm


In [14]:
import torch
from timm import create_model
from torchvision.transforms import Compose, Resize, Normalize, ToTensor
from tqdm import tqdm
import numpy as np

# Load a pretrained ViT model
vit_model = create_model('vit_base_patch16_224', pretrained=True)  # ViT-B/16
vit_model.reset_classifier(0)  # Remove the classification head
vit_model.eval()  # Set the model to evaluation mode

# Define preprocessing transformations
vit_preprocess = Compose([
    Resize((224, 224)),  # Resize to 224x224
    ToTensor(),  # Convert to PyTorch Tensor
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])


# Feature extraction function with tqdm
def extract_features_vit(loader, model, device="cpu"):
    model.to(device)
    features_list = []

    with torch.no_grad():
        for images in tqdm(loader, desc="Extracting Features"):
            images = images.to(device)
            outputs = model.forward_features(images)  # Extract features from the transformer encoder
            features_list.append(outputs.cpu().numpy())

    return np.vstack(features_list)  # Combine all feature vectors


train_features_vit = extract_features_vit(train_loader, vit_model, device)
val_features_vit = extract_features_vit(val_loader, vit_model, device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Extracting Features: 100%|██████████| 5/5 [00:35<00:00,  7.06s/it]


In [16]:
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

train_features_vit_flat = train_features_vit.reshape(train_features_vit.shape[0], -1)
val_features_vit_flat = val_features_vit.reshape(val_features_vit.shape[0], -1)

# SVC with ViT features
svc_vit = SVC(kernel="linear", random_state=42)
svc_vit.fit(train_features_vit_flat, metadata_train['melanoma'])
val_predictions_svc_vit = svc_vit.predict(val_features_vit_flat)
print("SVC (ViT) Classification Report:")
print(classification_report(metadata_validation['melanoma'], val_predictions_svc_vit))

# Random Forest with ViT features
rf_vit = RandomForestClassifier(n_estimators=100, random_state=42)
rf_vit.fit(train_features_vit_flat, metadata_train['melanoma'])
val_predictions_rf_vit = rf_vit.predict(val_features_vit_flat)
print("Random Forest (ViT) Classification Report:")
print(classification_report(metadata_validation['melanoma'], val_predictions_rf_vit))


SVC (ViT) Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.97      0.91       120
           1       0.71      0.33      0.45        30

    accuracy                           0.84       150
   macro avg       0.78      0.65      0.68       150
weighted avg       0.83      0.84      0.82       150

Random Forest (ViT) Classification Report:
              precision    recall  f1-score   support

           0       0.80      1.00      0.89       120
           1       0.00      0.00      0.00        30

    accuracy                           0.80       150
   macro avg       0.40      0.50      0.44       150
weighted avg       0.64      0.80      0.71       150



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [20]:
from sklearn.model_selection import GridSearchCV

# Dynamically generate parameter ranges
param_grid = {
    'C': np.logspace(-3, 2, 6),       # C: [0.001, 0.01, 0.1, 1, 10, 100]
    'kernel': ['linear', 'rbf'],
    'gamma': np.logspace(-3, 2, 6)    # gamma: [0.001, 0.01, 0.1, 1, 10, 100]
}

# Perform GridSearchCV
grid_search = GridSearchCV(
    estimator=SVC(kernel="linear", C=1, random_state=42),
    param_grid=param_grid,
    cv=3,                     # 3-fold cross-validation
    scoring='accuracy',
    verbose=2,
    n_jobs=-1                 # Use all processors
)

grid_search.fit(train_images,  train_labels)

# Best hyperparameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)


Fitting 3 folds for each of 72 candidates, totalling 216 fits
Best Parameters: {'C': 10.0, 'gamma': 0.001, 'kernel': 'rbf'}
Best Cross-Validation Score: 0.8140006573289932


In [19]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Define the parameter distribution
param_dist = {
    'C': np.logspace(-3, 2, 6),       # C: [0.001, 0.01, 0.1, 1, 10, 100]
    'kernel': ['linear', 'rbf'],
    'gamma': np.logspace(-3, 2, 6)    # gamma: [0.001, 0.01, 0.1, 1, 10, 100]
}

# RandomizedSearchCV
random_search = RandomizedSearchCV(svc_vit, param_distributions=param_dist, n_iter=10, cv=3, verbose=1, n_jobs=-1, random_state=42)
random_search.fit(train_features_vit_flat, metadata_train['melanoma'])

# Best parameters and score
print("Best parameters:", random_search.best_params_)
print("Best cross-validation score:", random_search.best_score_)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best parameters: {'kernel': 'linear', 'gamma': 0.09999999999999999, 'C': 0.001}
Best cross-validation score: 0.8039966503234869


In [21]:
from sklearn.metrics import classification_report

# GridSearchCV Classification Report
print("GridSearchCV Best Parameters:", grid_search.best_params_)
print("GridSearchCV Best Cross-Validation Score:", grid_search.best_score_)

# Predict using the best model from GridSearchCV
best_model_grid = grid_search.best_estimator_
val_predictions_grid = best_model_grid.predict(val_images)

# Generate classification report
report_grid = classification_report(val_labels, val_predictions_grid, target_names=["Non-Melanoma", "Melanoma"])
print("GridSearchCV Classification Report:")
print(report_grid)

# RandomizedSearchCV Classification Report
print("RandomizedSearchCV Best Parameters:", random_search.best_params_)
print("RandomizedSearchCV Best Cross-Validation Score:", random_search.best_score_)

# Predict using the best model from RandomizedSearchCV
best_model_random = random_search.best_estimator_
val_predictions_random = best_model_random.predict(val_features_vit_flat)

# Generate classification report
report_random = classification_report(metadata_validation['melanoma'], val_predictions_random, target_names=["Non-Melanoma", "Melanoma"])
print("RandomizedSearchCV Classification Report:")
print(report_random)


GridSearchCV Best Parameters: {'C': 10.0, 'gamma': 0.001, 'kernel': 'rbf'}
GridSearchCV Best Cross-Validation Score: 0.8140006573289932


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


GridSearchCV Classification Report:
              precision    recall  f1-score   support

Non-Melanoma       0.80      1.00      0.89       120
    Melanoma       0.00      0.00      0.00        30

    accuracy                           0.80       150
   macro avg       0.40      0.50      0.44       150
weighted avg       0.64      0.80      0.71       150

RandomizedSearchCV Best Parameters: {'kernel': 'linear', 'gamma': 0.09999999999999999, 'C': 0.001}
RandomizedSearchCV Best Cross-Validation Score: 0.8039966503234869
RandomizedSearchCV Classification Report:
              precision    recall  f1-score   support

Non-Melanoma       0.85      0.97      0.91       120
    Melanoma       0.71      0.33      0.45        30

    accuracy                           0.84       150
   macro avg       0.78      0.65      0.68       150
weighted avg       0.83      0.84      0.82       150



In [26]:
# Export GridSearchCV Best Model
grid_model_path = "svc_gridsearch_model.pkl"
joblib.dump(best_model_grid, grid_model_path)
print("GridSearchCV best model saved as 'svc_gridsearch_model.pkl'")
files.download(grid_model_path)

GridSearchCV best model saved as 'svc_gridsearch_model.pkl'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
random_model_path = "svc_randomsearch_model.pkl"
joblib.dump(best_model_random, random_model_path)
print("RandomizedSearchCV best model saved as 'svc_randomsearch_model.pkl'")
files.download(random_model_path)

RandomizedSearchCV best model saved as 'svc_randomsearch_model.pkl'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
vit_model_path = "svc_vit_model.pkl"
joblib.dump(svc_vit, vit_model_path)
print("SVC (ViT) model saved as 'svc_vit_model.pkl'")
files.download(vit_model_path)


SVC (ViT) model saved as 'svc_vit_model.pkl'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
svm_model_path = "svm_model.pkl"
joblib.dump(svm, svm_model_path)
print("SVM model saved as 'svm_model.pkl'")
files.download(svm_model_path)

SVM model saved as 'svm_model.pkl'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>